### Fashion MNIST experiment 

In [ ]:
import numpy as np

from keras.layers import Dense, Activation, Flatten
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout

import matplotlib.pyplot as plt

from utils import mnist_reader

%matplotlib inline

#### 1), load data 

In [ ]:
X_train, y_train = mnist_reader.load_mnist('./data/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('./data/data/fashion', kind='t10k')

In [ ]:
X_train = (X_train.reshape((X_train.shape[0], 28, 28, 1)) / 255.0).astype(np.float32)
X_test = (X_test.reshape((X_test.shape[0], 28, 28, 1)) / 255.0).astype(np.float32)

In [ ]:
def id_to_onehot(y, size):
    onehot = np.zeros(shape=(y.shape[0], size))
    for i in range(0, y.shape[0]):
        onehot[i][y[i]] = 1
    return onehot

def show_img(img, zoom=4, dpi=80):
    w = img.shape[0]
    h = img.shape[1]
    plt.figure(figsize=(w*zoom/dpi, h*zoom/dpi), dpi=dpi)
    plt.axis('off')
    plt.imshow(img, cmap=plt.get_cmap('gray'))
    plt.show()
    return

In [ ]:
show_img(X_train[234].reshape(28, 28))

In [ ]:
y_train = id_to_onehot(y_train, 10)
y_test = id_to_onehot(y_test, 10)

#### 2), build model 

In [ ]:
m = Sequential()

m.add(Flatten(input_shape=(28, 28, 1)))
m.add(Dense(800))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dense(10))
m.add(BatchNormalization())
m.add(Activation('softmax'))

m.summary()

In [ ]:
m.compile(optimizer='Adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

m.fit(X_train,
      y_train,
      validation_data=(X_test, y_test),
      batch_size=32,
      epochs=10)

In [ ]:
evaluation_test = m.evaluate(X_train, y_train, verbose=0)

In [ ]:
print('loss: {0}, accuracy: {1}'.format(evaluation_test[0], evaluation_test[1]))